In [1]:
%%html
<link rel="stylesheet" type="text/css" href="theme/sixty_north.css">

In [2]:
%run unittest_magics.py
from hypothesis import given

# `Hypothesis`
## Property-based testing for Python

[hypothesis.works](http://hypothesis.works/)

**"...the larger purpose of Hypothesis is to drag the world kicking and screaming into a new
and terrifying age of high quality software."**  

*--David MacIver, Hypothesis manifesto*

* Basic idea: express "hypotheses"/invariants about your code and let `hypothesis` try to disprove them. Fundamentally, it searches for sets input to your test functions which trigger assertions in the tests.
* Strategies: An object with methods that describe how to generate and simplify certain kinds of values.
  * Hypothesis simplifies when it finds breaking inputs
  * primitive types
  * lists, tuples
  * choice, sample_from
  * stream
  * mapping, filtering
  * flatmap
  * recursive
  * composite
  * draw (interactive strategy drawing)
* @given
  * How hypothesis mathes givens to arguments
* @example
* assume (nan example)
* settings
  * e.g. max_examples, verbosity
  * profiles
* note
* event
* test statistics
* executors
* find() for data exploration
* extras: django, datetime, numpy, etc.
* database
* stateful testing

# Basic concept

1. Define invariants or "hypotheses" about your code
2. Tell `hypothesis` how to parameterize your tests
3. `hypothesis` searches for parameters that disprove your invariants

# A motivating example

This simple GCD function will help us see the fundamental value of `hypothesis`.

In [3]:
def gcd(m, n):
    while m and n:
        m, n = max(m, n) - min(m, n), min(m, n) 
    return max(m, n)

With example-based testing we'd throw a lot of examples at the function and look for errors.

In [4]:
%%unittest_run

import itertools

class Tests(unittest.TestCase):
    def test_zero_remainder(self):
        for m,n in itertools.product(range(1, 1000), range(1, 100)):
            d = gcd(m, n)
            self.assertEqual(m % d, 0)
            self.assertEqual(n % d, 0)
            
    def test_gcd_is_greatest(self):
        for m,n in itertools.product(range(1, 100), range(1, 100)):
            d = gcd(m, n)
            for other in range(d + 1, 100):
                self.assertTrue(m % other or n % other)

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 2


In [5]:
%%unittest_run

from hypothesis.strategies import assume, integers

class Tests(unittest.TestCase):
    @given(integers(min_value=1, max_value=2000), 
           integers(min_value=1, max_value=2000))
    def test_zero_remainder(self, m, n):
        d = gcd(m, n)
        self.assertEqual(m % d, 0)
        self.assertEqual(n % d, 0)
            
    @given(integers(min_value=1, max_value=2000), 
           integers(min_value=1, max_value=2000), 
           integers(min_value=1, max_value=2000))
    def test_gcd_is_greatest(self, m, n, other):
        d = gcd(m, n)
        assume(other > d)
        self.assertTrue(m % other or n % other)

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 2


# Strategies
## Define how to generate and simplify input data

In [11]:
import hypothesis.strategies as st

strategies = [
    st.integers(),
    st.text(),
    st.fractions(),
    st.lists(st.booleans())
]

[s.example() for s in strategies]

[1345069054821528362791176458764103664,
 '\x1c\U0009bf8d\U001076f0\U0009b0f3\x16𘜎)啒\x1c)',
 Fraction(-25317944005634928601889704397684258051, 2733),
 [False,
  False,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  False,
  True,
  False,
  True,
  False,
  True]]

# Primitive strategies
## Generate all of the Python primitive types (and then some)

* `floats`, `integers`, `booleans`, `None`
* `text`, `binary`
* `uuids`, `fractions`

You will often use these directly, as well as part of more complex strategies

In [15]:
%%unittest_run

from hypothesis import given

class Tests(unittest.TestCase):
    @given(st.uuids(), st.fractions(), 
           st.binary(min_size=2, max_size=10))
    def test_primitive_strategies(self, uuid, fraction, binary):
        print(uuid, fraction, binary)
        self.assertTrue(False)

e3e70682-c209-4cac-629f-6fbed82c07cd 0 b'\x00\x00'
cd613e30-d8f1-6adf-91b7-584a2265b1f5 0 b'\x00\x00'
d95bafc8-f2a4-d27b-dcf4-bb99f4bea973 0 b'\x00\x00'
21636369-8b52-9b4a-97b7-50923ceb3ffd 0 b'\x00\x00'
b8a1abcd-1a69-16c7-4da4-f9fc3c6da5d7 0 b'\x00\x00'
e3e70682-c209-4cac-629f-6fbed82c07cd 309489732187827938369994752 b'\x00\x00'
fcf537eb-4fe7-cc04-a12a-2e357650eabe 309489732187827938369994752 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 309489732187827938369994762 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 309489750634572012079546378 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 310698676454186641254252554 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 19411210514671216897059522084874 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 3751691490236635632253727934513162 b'\x00\x00'
b08583df-056e-417f-2f57-643538ddc114 3751691763455401944918039310893066 b'\x00\x00'
7c754da8-a79a-af68-21be-20058d56bf93 3751691763456398202518958198620500 b'\x00\x00'
e3e70682-c209-4cac-629f-6f

# Container strategies
## Generate lists, dictionaries, and tuples using other strategies

* `lists`
* `dictionaries`, `fixeddictionaries`
* `sets`, `frozensets`
* `streaming`
* `tuples`

In [16]:
%%unittest_run

class Tests(unittest.TestCase):
    @given(st.dictionaries(st.integers(), st.text()), 
           st.tuples(st.floats()))
    def test_container_strategies(self, d, s):
        print(d, s)
        self.assertEqual(len(d), len(s))

{} (0.0,)
{} (-0.0,)
{} (1.0,)
{} (0.0,)
{} (0.0,)
{5752775138963000729769343794433: '0'} (0.0,)
{} (-1.0,)
{} (0.0,)
Falsifying example: test_container_strategies(self=<__main__.Tests testMethod=test_container_strategies>, d={}, s=(0.0,))
{} (0.0,)
You can add @seed(302934307671667531413257853548643485645) to this test to reproduce this failure.
FAILURES: 1 ERRORS: 0 SKIPPED: 0 TOTAL: 1
f: test_container_strategies (__main__.Tests)


# Mapping and filtering strategies
## Declarative modification and constraint of strategies

* `<any strategy>.map(func)`: Map a function over a strategy
* `<any strategy>.filter(func)`: Keep only matching values from a strategy

In [17]:
%%unittest_run

class Tests(unittest.TestCase):
    @given(st.integers().map(lambda x: x * 2), 
           st.integers().filter(lambda x: x % 2 == 0))
    def test_even_minus_even_is_even(self, x, y):
        self.assertTrue((x - y) % 2 == 0)

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


## `<strategy>.flatmap()` 
## Use one example to generate further strategies

In [22]:
s = st.integers(min_value=1, max_value=3).flatmap(
    lambda n: st.lists(st.lists(st.integers(), 
                                min_size=n, 
                                max_size=n)))
s.example()

[[-25909775738913710926634836742338635379,
  -165056919985583205381283874825657424602],
 [90023277389163682733307319860207328474,
  22460900724744407855333819913302342200],
 [125772708953927921071741597382378939575,
  -141193200898304880067957975728302279953],
 [-18241233507216142366104434450720748155,
  -4459376790664378858124194985086297761],
 [-82408998998961475073199653815779057631,
  134304278128926740089687675485946972189],
 [-151821323887669877652939342192476160853,
  6794280379345776912713999362659688091]]

# Other strategy goodies

* `<strategy>.recursive()`: Generate recursive data structures, e.g. JSON data
* `hypothesis.strategies.composite`: Combine strategies in arbitrary ways, e.g. lists plus an index
* `hypothesis.strategies.data`: Interactively generate values from strategies

# The `given` decorator
## Defines how strategies are associated with test function parameters

From the `hypothesis` documentation:

* You may pass any keyword argument to `given`.
* Positional arguments to `given` are equivalent to the rightmost named
  arguments for the test function.
* Positional arguments may not be used if the underlying test function has
  varargs or arbitrary keywords.
* Functions tested with `given` may not have any defaults.


In [23]:
# All of these tests are equivalent

class Tests(unittest.TestCase):
    @given(x=st.integers(), y=st.floats())
    def test_example1(self, x, y):
        print(x, y)
    
    @given(st.integers(), st.floats())
    def test_example2(self, x, y):
        print(x, y)
    
    @given(st.floats(), x=st.integers())
    def test_example3(self, x, y):
        print(x, y)

# The `example` decorator
## Instruct hypothesis to use some specific examples

Examples will be executed before strategy-defined inputs.

Useful for testing known edge-cases or other trouble spots.

In [24]:
%%unittest_run

from hypothesis import example, settings
from string import ascii_letters

class Tests(unittest.TestCase):
    @given(st.text(alphabet=ascii_letters), 
           st.text(alphabet=ascii_letters))
    @example('', '')
    @example('foo', 'bar')
    @settings(max_examples=1)
    def test_levenshtein(self, m, n):
        print('inputs:', m, n)

inputs:  
inputs: foo bar
inputs:  
FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


# `assume()`
## Indicate that failures under certain conditions should be ignored

`assume()` lets you tell `hypothesis` that the current examples should be considered "bad examples". The test will be aborted with an exception but `hypothesis` will not consider it a failure.

In [25]:
%%unittest_run

from hypothesis import given, assume
from math import isnan

# adapted from hypothesis documentation
class Tests(unittest.TestCase):
    @given(st.floats())
    def test_negation_is_self_inverse_for_non_nan(self, x):
        assume(not isnan(x))
        self.assertEqual(x, -(-x))

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


# Stateful testing
## Find bad states by automatically driving your API

* Define API call relationships with a state machine
* `hypothesis` can still help generate input data
* Use assertions and preconditions to check your program state

This is new and semi-public, but is considered ready for general use.

# Further topics

* `settings`: get and set values that control how `hypothesis` behaves
* `note`: additional diagnostics for final test runs
* `event`: inject messages into `hypothesis`'s event log
* executors: control the test execution context
* `find()`: explore example sets for those that meet certain criteria
* extras: django, datetime, numpy, etc.
* database: `hypothesis` stores/caches information in a directory structure